# Quick Start Guide

Import *qcodes* from the *zhinst* namespace as follows:


In [1]:
import numpy as np
import matplotlib.pyplot as plt

import qcodes as qc
import zhinst.qcodes as ziqc

## Initialize the Device

*zhinst-qcodes* provides instrument drivers for multiple different *Zurich Instruments* devices. It includes drivers for

* HDAWG
* UHFQA
* UHFLI
* MFLI

All of the instrument drivers available in *zhinst-qcodes* share some basic device properties. Every device is identified by a `name` and a `serial` number. The `name` is free for the user to chose and only for their convenience. The `serial` however, is a unique identifier for every *Zurich Instruments* device. The serial number can be found on the back of panel of instrument and is of the format `DEV1234`. In addition, the drivers needs to know what `interface` the device uses and where to find the data server (`host`). The value for the interface defaults to a connection via Ethernet (`1GbE`) and the host address is `localhost` unless specified otherwise.

As a first example, we now initialize the instrument driver for a *HDAWG*. We name it `hdawg1` and we know that our device's serial number is `dev8138`.

In [2]:
hdawg = ziqc.HDAWG("hdawg1", "dev8138", interface="1gbe", host="10.42.0.226")

Successfully connected to data server at 10.42.0.2268004 api version: 6
Successfully connected to device DEV8138 on interface 1GBE
Connected to: Zurich Instruments HDAWG (serial:dev8138, firmware:66245) in 3.26s


Now the device is connected and we are ready to go!

In [3]:
hdawg.get_idn()

{'vendor': 'Zurich Instruments',
 'model': 'HDAWG',
 'serial': 'dev8138',
 'firmware': 66245}

## Access the Device's Nodetree  

The driver's *nodetree* is a data structure that allows the user to access all the settings and data on the device. The settings are highly structured into logical groups, enumerations and options. In QCoDeS they are represented as *submodules*.

For example, all of the device's signal inputs and all of its signal outputs are grouped together or the HDAWG's 8 sine generators are grouped into the submodule `sines` (a `ChannelList`) that are then enumerated from `0 - 7`. Each `Node` in the nodetree can have other submodules or QCoDeS `Parameters` as attributes. The `Parameters` are the 'leaves' of the tree data structure and represent individual settings or data on the instrument.

```
HDAWG
│
└─── sigouts             <--- ChannelList
│    └─── 0              <--- InstrumentChannel 
│    │    └─── on        <--- Parameter
│    │    └─── range 
│    │    └─── direct
│    │    └─── offset
│    │    └─── ...
│    └─── 1  
│
└─── oscs 
│    └─── 0
│    │    └─── freq
│    └─── 1
│    └─── 2
│    └─── ...
│
└─── sines
│    └─── ...
│
└─── awgs
│    └─── 0
│    └─── 1
│    └─── 2
│    └─── 4
│
└─── ...     
```

Enumerated nodes such as the 8 sine-generators or 16 oscillators of the *HDAWG* are grouped together in `ChannelLists`. For example the node of the first sine-generator would be accessed via `hdawg.sines[0]`. 

In a *Jupyter* notebook or a console it is easy to navigate through the nodetree.

In [6]:
print([k for k in hdawg.submodules.keys()])
print([k for k in hdawg.parameters.keys()])

['awgs', 'stats', 'oscs', 'status', 'sines', 'dios', 'system', 'sigouts', 'triggers', 'features', 'cnts']
['IDN', 'clockbase']


In [7]:
hdawg.sines[0:2]

ChannelList(<HDAWG: hdawg1>, ZINode, (<ZINode: hdawg1_sines0 of HDAWG: hdawg1>, <ZINode: hdawg1_sines1 of HDAWG: hdawg1>))

At the end of the tree, the leaves, are QCoDeS `Parameters`. In case you wonder what a certain parameter does, you can print an insightful description.

In [8]:
print(hdawg.oscs[0].freq.__doc__)

* `Node`: /DEV8138/OSCS/0/FREQ
* `Description`: Frequency control for each oscillator.
* `Properties`: Read, Write, Setting
* `Type`: Double
* `Unit`: Hz


Parameter class:

* `name` freq
* `label` freq
* `unit` Hz
* `vals` None


The parameters are callable to set and get the instrument values.

In [7]:
# set oscillator frequency to 100 MHz
hdawg.nodetree.oscs[0].freq(100e6)

# what frequency is the oscillator set to now?
hdawg.nodetree.oscs[0].freq()

99999999.99999432

The instrument driver's submodules that represent the nodetree are created whenever it connects the device to the data server. This ensures that the available nodes and parameters in the nodetree are always up to date with the latest version of the device's firmware. 

Depending on the device, the instruemnt drivers also have submodules other than the ones in the nodetree. Those are called *Modules* and depend on the type of device at hand and the options installed on it. Modules can for example be the *Data Acquisition Module* or the *Sweeper Module*. The devices with AWG Sequencers such as the *HDAWG*, *UHFQA* and the *UHFLI* (if option is installed) feature *AWG Modules*. Each *AWG Module* controls two chanels and features high-level functionality for programming sequences of waveforms and generating IQ-Modulation signals.

For more information, see the detailed documentation of the `zhinst-toolkit` and the following examples. 